<a href="https://colab.research.google.com/github/SurriyaGokul/CrossDepth-NeRF/blob/main/Output_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ------------------------------------
# Install required packages
# ------------------------------------
!pip install moviepy lpips ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

# ------------------------------------
# Imports
# ------------------------------------
import os
import torch
import clip
from PIL import Image
import moviepy.editor as mp
import cv2
import numpy as np
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

In [1]:
import os
import cv2
from tqdm import tqdm
import moviepy.editor as mp

input_video_path = "/content/output.mp4"
frames_dir = "/content/cropped_frames"
output_video_path = "/content/cropped_output.mp4"

# Extract frames
os.makedirs(frames_dir, exist_ok=True)

clip = mp.VideoFileClip(input_video_path)
fps = clip.fps

for i, frame in enumerate(clip.iter_frames()):
    h, w, _ = frame.shape
    cropped = frame[h//2: , :]   # Keep bottom half
    cv2.imwrite(os.path.join(frames_dir, f"frame_{i:04d}.png"), cv2.cvtColor(cropped, cv2.COLOR_RGB2BGR))

print(f"✅ Frames extracted and cropped to bottom half. Total frames: {i+1}")

# ------------------------------------
# Recreate video
# ------------------------------------
img_files = sorted([os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith('.png')])
if img_files:
    first_img = cv2.imread(img_files[0])
    height, width, _ = first_img.shape

    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    for img_path in tqdm(img_files):
        img = cv2.imread(img_path)
        out.write(img)
    out.release()
    print(f"✅ Cropped video saved to: {output_video_path}")
else:
    print("❌ No images found to create video.")


  if event.key is 'enter':



✅ Frames extracted and cropped to bottom half. Total frames: 240


100%|██████████| 240/240 [00:01<00:00, 186.12it/s]

✅ Cropped video saved to: /content/cropped_output.mp4


In [3]:
import os
import torch
import clip
import numpy as np
import moviepy.editor as mp
from PIL import Image
import lpips
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

device = "cuda" if torch.cuda.is_available() else "cpu"


In [14]:
import os
import numpy as np
import torch
import clip
from PIL import Image
import moviepy.editor as mp
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Paths to videos
video_paths = {
    "output": "/content/cropped_output.mp4",
    "trueoutput": "/content/trueoutput.mp4",
}

# Extract frames from video and save as images
def extract_frames(video_path, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    clip_video = mp.VideoFileClip(video_path)
    for i, frame in enumerate(clip_video.iter_frames()):
        img = Image.fromarray(frame)
        img.save(os.path.join(out_dir, f"frame_{i:04d}.png"))
    return sorted(os.listdir(out_dir))

# Load CLIP model
clip_model, preprocess = clip.load("ViT-B/32", device=device)

# Compute CLIP score between image and text prompt
def compute_clip_score(frame_path, text_prompt="polar bear"):
    image = preprocess(Image.open(frame_path)).unsqueeze(0).to(device)
    text = clip.tokenize([text_prompt]).to(device)
    with torch.no_grad():
        image_features = clip_model.encode_image(image)
        text_features = clip_model.encode_text(text)
        score = (image_features @ text_features.T).squeeze().cpu().item()
    return score

# Extract frames for trueoutput first
true_dir = "/content/frames_trueoutput"
true_files = extract_frames(video_paths["trueoutput"], true_dir)

metrics_results = {}

# Process each video
for name, vpath in video_paths.items():
    frame_dir = f"/content/frames_{name}"
    if name != "trueoutput":
        frame_files = extract_frames(vpath, frame_dir)
    else:
        frame_files = true_files

    metrics_results[name] = {
        "clip_scores": [],
    }

    # Compute CLIP score for each frame
    for f in tqdm(frame_files):
        frame_path = os.path.join(frame_dir, f)
        clip_s = compute_clip_score(frame_path)
        metrics_results[name]["clip_scores"].append(clip_s)

# Print mean CLIP score for each video
for name in metrics_results:
    print(f"\nResults for {name}:")
    cs = np.mean(metrics_results[name]["clip_scores"])
    print(f"Mean CLIP score: {cs:.3f}")



Results for output:
Mean CLIP score: 30.008

Results for trueoutput:
Mean CLIP score: 28.068


In [9]:
!pip install pyiqa --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.2/276.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
import os
import torch
import numpy as np
import pyiqa
from PIL import Image
import torchvision.transforms as T
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Directory with extracted frames
output_frames_dir = "/content/frames_output"

# Image preprocessing
transform_tensor = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
])

niqe_scores = []

# Load NIQE metric
niqe_metric = pyiqa.create_metric('niqe').to(device)

# Get sorted frame file names
output_files = sorted(os.listdir(output_frames_dir))

# Compute NIQE score for each frame
for f in tqdm(output_files, desc="Computing NIQE"):
    output_path = os.path.join(output_frames_dir, f)
    img_out = Image.open(output_path).convert("RGB")
    img_out_tensor = transform_tensor(img_out).unsqueeze(0).to(device)

    niqe_score = niqe_metric(img_out_tensor).item()
    niqe_scores.append(niqe_score)

# Calculate mean NIQE
mean_niqe = np.nanmean(niqe_scores)
print(f"Mean NIQE: {mean_niqe:.3f}")


Mean NIQE: 4.130
